In [ ]:
import os
import zipfile
import glob

# 1. 既存データのクリーニング（再実行時用）とフォルダ作成
if os.path.exists('vtu'):
    import shutil
    shutil.rmtree('vtu')
os.makedirs('vtu', exist_ok=True)

# 2. データのダウンロード
# raw URLを使用し、HTMLではなくZIP本体を取得します
DATA_URL = "https://github.com/WOCae/R211/raw/main/vtu.zip"
!wget -O data_files.zip "{DATA_URL}" -q

# 3. 解凍 (-j オプションでフォルダ階層を無視して vtu/ 直下に展開)
!unzip -jo data_files.zip -d vtu

# 4. ファイルリストの取得
all_vtu_files = sorted(glob.glob("vtu/*.vtu"))
print(f"準備完了: {len(all_vtu_files)} 個のファイルを 'vtu' フォルダに展開しました。")

In [ ]:
!pip install torch numpy
!pip install pyvista pyvistaqt
!pip install tqdm
!pip install vtk


In [ ]:
import torch
import torch.nn as nn
import torch.fft as fft

In [ ]:
# --- 設定の一括管理（メモリ節約モード） ---
class Config:
    # 3Dグリッド解像度
    nx, ny, nz = 24, 24, 24

    # 荷重の正規化係数
    load_norm = 10000.0

    # 物理量のスケール（正規化・復元用）
    scales = {
        'ux': 1e-4, 'uy': 1e-4, 'uz': 1e-4, 
        'mises': 1e7,
        'disp': 1e-4,
        'stress': 1e7
    }

    # デバイス設定
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # 可視化・復元用の定数
    NORM_DISP = 1e-4
    NORM_STRESS = 1e7
    LOAD_NORM_FACTOR = 10000.0

    # 物理座標の範囲（必要なら追加）
    x_range = (0.0, 10.0)
    y_range = (0.0, 10.0)
    z_range = (0.0, 100.0)


print(f"Using device: {Config.device}")
print(f"Grid size: {Config.nx}x{Config.ny}x{Config.nz}")

In [ ]:
class SpectralConv3d(nn.Module):
    def __init__(self, in_channels, out_channels, modes_x, modes_y, modes_z):
        super().__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.modes_x = modes_x
        self.modes_y = modes_y
        self.modes_z = modes_z

        # 3Dの重みパラメータ (kx, ky, kz)
        self.weight_real = nn.Parameter(
            torch.randn(in_channels, out_channels, modes_x, modes_y, modes_z) * 0.1
        )
        self.weight_imag = nn.Parameter(
            torch.randn(in_channels, out_channels, modes_x, modes_y, modes_z) * 0.1
        )

    def compl_mul3d(self, input, weight_real, weight_imag):
        # (batch, in_channel, x, y, z) x (in_channel, out_channel, x, y, z) -> (batch, out_channel, x, y, z)
        real = torch.einsum("bixyz,ioxyz->boxyz", input.real, weight_real) - \
               torch.einsum("bixyz,ioxyz->boxyz", input.imag, weight_imag)
        imag = torch.einsum("bixyz,ioxyz->boxyz", input.real, weight_imag) + \
               torch.einsum("bixyz,ioxyz->boxyz", input.imag, weight_real)
        return torch.complex(real, imag)

    def forward(self, x):
        batchsize, channels, nx, ny, nz = x.shape
        # 3D FFT
        x_ft = fft.rfftn(x, dim=[-3, -2, -1], norm="ortho")

        out_ft = torch.zeros(
            batchsize, self.out_channels, x_ft.size(-3), x_ft.size(-2), x_ft.size(-1),
            dtype=torch.cfloat, device=x.device
        )

        # 低周波成分のみ抽出して複素行列乗算
        kx, ky, kz = self.modes_x, self.modes_y, self.modes_z
        out_ft[:, :, :kx, :ky, :kz] = self.compl_mul3d(
            x_ft[:, :, :kx, :ky, :kz],
            self.weight_real, self.weight_imag
        )

        # 3D 逆FFT
        return fft.irfftn(out_ft, s=(nx, ny, nz), dim=[-3, -2, -1], norm="ortho")



class FNO3d(nn.Module):
    def __init__(self, in_channels=2, out_channels=4, width=32, modes=8):
        super().__init__()
        self.width = width
        self.fc_in = nn.Linear(in_channels, width)

        # 3Dフーリエ層と3D残差接続
        self.convs = nn.ModuleList([SpectralConv3d(width, width, modes, modes, modes) for _ in range(4)])
        self.ws = nn.ModuleList([nn.Conv3d(width, width, 1) for _ in range(4)])

        self.fc_out1 = nn.Linear(width, 128)
        self.fc_out2 = nn.Linear(128, out_channels)
        self.act = nn.GELU()

    def forward(self, x):
        # x shape: (batch, nx, ny, nz, in_channels)
        x = self.fc_in(x) 
        x = x.permute(0, 4, 1, 2, 3) # (batch, width, nx, ny, nz)

        for conv, w in zip(self.convs, self.ws):
            x1 = conv(x)
            x2 = w(x)
            x = self.act(x1 + x2)

        x = x.permute(0, 2, 3, 4, 1) # (batch, nx, ny, nz, width)
        x = self.act(self.fc_out1(x))
        x = self.fc_out2(x) # (batch, nx, ny, nz, out_channels)
        return x

In [ ]:
import ipywidgets as widgets
from IPython.display import display

# --- 対話型設定ウィジェットの作成 ---

# 1. ファイル選択 (デフォルトで全選択)
file_selector = widgets.SelectMultiple(
    options=[(os.path.basename(f), f) for f in all_vtu_files],
    value=all_vtu_files,
    description='学習データ:',
    layout={'width': '600px', 'height': '180px'}
)

# 2. 実行モード選択
mode_dropdown = widgets.Dropdown(
    options=[('新規学習 (Train)', 'train'), ('学習済みモデル読込 (Load)', 'load')],
    value='train',
    description='モード:',
)

print("【設定】以下のリストから使用するデータとモードを選択してください。")
display(widgets.VBox([
    widgets.Label("1. データ選択 (Ctrl/Shiftで複数選択可):"),
    file_selector,
    widgets.Label("2. 実行モード:"),
    mode_dropdown
]))

In [ ]:
import pyvista as pv
import numpy as np


def load_case(vtu_path):
    """1つのVTUファイルから節点座標, 変位, ミーゼス応力を読み込む"""
    mesh = pv.read(vtu_path)

    # 節点座標 (N, 3)
    pts = mesh.points.astype(np.float32)

    # 変位ベクトル (N, 3)
    disp = mesh.point_data["Displacement"].astype(np.float32)

    # von Mises 応力 (N,)
    mises = mesh.point_data["von Mises Stress"].astype(np.float32)

    return pts, disp, mises

In [ ]:
import numpy as np
from scipy.interpolate import NearestNDInterpolator

# --- 設定値の修正案 ---
# class Config: ... ← この定義は削除してください（先頭で定義済み）

def interpolate_to_grid_3d_optimized(pts, disp, mises):
    """
    メモリ効率の良い最近傍補間で3Dグリッド化を行う
    """
    points = pts[:, :3]
    
    # グリッド作成 (indexing='ij'で整列)
    # 物理的な範囲に合わせてlinspaceを作成
    gx = np.linspace(Config.x_range[0], Config.x_range[1], Config.nx, dtype=np.float32)
    gy = np.linspace(Config.y_range[0], Config.y_range[1], Config.ny, dtype=np.float32)
    gz = np.linspace(Config.z_range[0], Config.z_range[1], Config.nz, dtype=np.float32)
    
    grid_x, grid_y, grid_z = np.meshgrid(gx, gy, gz, indexing='ij')
    
    # ターゲット座標一覧
    target_points = np.stack([grid_x.ravel(), grid_y.ravel(), grid_z.ravel()], axis=-1)

    # 教師データ (変位3成分 + ミーゼス応力)
    values = np.column_stack([disp[:,0], disp[:,1], disp[:,2], mises])
    
    # ★重要: LinearNDInterpolatorではなくNearestを使用（メモリ対策）
    interp = NearestNDInterpolator(points, values)
    interpolated = interp(target_points)
    
    # グリッド形状に戻す (nx, ny, nz, 4)
    result_grid = interpolated.reshape(Config.nx, Config.ny, Config.nz, 4)
    
    # 成分ごとに分離
    ux = result_grid[..., 0]
    uy = result_grid[..., 1]
    uz = result_grid[..., 2]
    ms = result_grid[..., 3]
    
    return ux, uy, uz, ms

def make_inp_out(pts, disp, mises, load_value):
    # 1. 3Dデータへの補間
    ux, uy, uz, ms = interpolate_to_grid_3d_optimized(pts, disp, mises)
    
    # 2. 入力データの作成 (Input Channels)
    # ここで「どこが固定で、どこが荷重か」をCNNに教えます
    
    # Ch0: Geometry Mask (今回は全体が梁なので全て1)
    # 本来は air=0, material=1
    geo_mask = np.ones((Config.nx, Config.ny, Config.nz), dtype=np.float32)
    
    # Ch1: Boundary Condition (Fixed) -> Z=0 の面を固定と仮定
    # index 0 のスライスだけ 1 にする
    fix_mask = np.zeros_like(geo_mask)
    fix_mask[:, :, 0] = 1.0 
    
    # Ch2: Boundary Condition (Load) -> Z=100 (index -1) の面を荷重負荷と仮定
    load_mask = np.zeros_like(geo_mask)
    load_mask[:, :, -1] = 1.0 # 最後の層
    
    # Ch3: Load Value -> 荷重値を埋め込む
    # 方法A: 全体に埋める（CNNがGeometryと組み合わせて判断）
    # 方法B: Load Maskの場所だけに値を入れる
    # ここでは学習のしやすさから「全体に正規化した値」を入れるパターンを推奨します
    # ※ load_value は事前に最大値などで割って 0.0~1.0 や -1.0~1.0 に正規化することを推奨
    val_channel = np.full_like(geo_mask, load_value, dtype=np.float32)
    
    # 入力を結合: (4, nx, ny, nz)
    inp = np.stack([geo_mask, fix_mask, load_mask, val_channel], axis=0)
    
    # 3. 出力データの作成（正規化してからスタック）
    #   変位: 1e-4 で割って O(1)
    #   応力: 1e7 で割って O(1)
    ux_n = ux / Config.scales['disp']
    uy_n = uy / Config.scales['disp']
    uz_n = uz / Config.scales['disp']
    ms_n = ms / Config.scales['stress']

    out = np.stack([ux_n, uy_n, uz_n, ms_n], axis=0)
    
    return inp, out

In [ ]:
import glob
import os
from tqdm import tqdm
import torch
from torch.utils.data import TensorDataset, DataLoader

# --- 【変更点】ウィジェットの選択結果を取得 ---
vtu_files = list(file_selector.value)
current_mode = mode_dropdown.value

if len(vtu_files) == 0:
    raise ValueError("エラー: ファイルが選択されていません。上のセルでデータを選択してください。")

print(f"モード: {current_mode}")
print(f"処理対象ファイル数: {len(vtu_files)}")

# ---------------------------------------------

inputs = []  # (nx, ny, nz, 4)
outputs = [] # (nx, ny, nz, 4)

print("Processing VTU -> 3D grids...")
for vtu_path in tqdm(vtu_files, total=len(vtu_files)):
    # ファイル名から荷重値を取得: 例 vtu/2_-2000.vtu -> -2000
    stem = os.path.splitext(os.path.basename(vtu_path))[0]
    try:
        # ファイル名が "ケース番号_荷重" 形式でない場合への対応
        if "_" in stem:
            _, load_str = stem.split("_", 1)
            load_val = float(load_str)
        else:
            # 暫定対応：ファイル名から数値が取れない場合はパススキップ等の処理が必要
            # ここではファイル名末尾の数値を拾うなどのロジックが必要になりますが、
            # 配布データ形式に従い "_" 分割で進めます
            raise ValueError("Underscore not found")
    except Exception:
         # データセットによっては "data_1000.vtu" のような形式もあるため柔軟に
        import re
        nums = re.findall(r'-?\d+', stem)
        if nums:
            load_val = float(nums[-1])
        else:
            raise ValueError(f"Could not parse load from file name: {stem}")

    # 1. VTU読み込み
    pts, disp, mises = load_case(vtu_path)

    # 2. 荷重を正規化
    load_norm = load_val / Config.load_norm

    # 3. 3Dグリッドへ補間し、入力/出力を作成
    inp, out = make_inp_out(pts, disp, mises, load_norm)

    # (4, nx, ny, nz) -> (nx, ny, nz, 4) に変換
    inp = np.moveaxis(inp, 0, -1).astype(np.float32)
    out = np.moveaxis(out, 0, -1).astype(np.float32)

    inputs.append(inp)
    outputs.append(out)

# NumPy -> Torch (B, nx, ny, nz, C)
X_all = np.stack(inputs, axis=0)
Y_all = np.stack(outputs, axis=0)

X_torch = torch.from_numpy(X_all).float()
Y_torch = torch.from_numpy(Y_all).float()

print(f"Input shape:  {X_torch.shape}")
print(f"Output shape: {Y_torch.shape}")

# データセット/ローダ
dataset = TensorDataset(X_torch, Y_torch)
loader = DataLoader(dataset, batch_size=1, shuffle=True)

In [ ]:
import torch

# モデルの初期化
model = FNO3d(
    in_channels=4,   # [geo, fix, load_mask, load_value]
    out_channels=4,  # [ux, uy, uz, mises]
    width=32,
    modes=8,
).to(Config.device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn = torch.nn.MSELoss()

# --- 【変更点】モードによる分岐 ---
MODEL_PATH = 'fem_surrogate_model.pth'

if current_mode == 'load':
    if os.path.exists(MODEL_PATH):
        print(f"保存済みモデル '{MODEL_PATH}' を読み込みます...")
        checkpoint = torch.load(MODEL_PATH, map_location=Config.device)
        model.load_state_dict(checkpoint['model_state_dict'])
        print("読み込み完了。学習をスキップします。")
    else:
        print(f"警告: '{MODEL_PATH}' が見つかりません。新規学習モードに切り替えます。")
        current_mode = 'train'

if current_mode == 'train':
    print("Start 3D FNO training...")
    for epoch in range(50):
        model.train()
        total_loss = 0.0

        for xb, yb in loader:
            xb = xb.to(Config.device)
            yb = yb.to(Config.device)

            pred = model(xb)
            loss = loss_fn(pred, yb)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        avg_loss = total_loss / len(loader)
        if (epoch + 1) % 5 == 0:
            print(f"Epoch {epoch+1:3d} / 50, Loss = {avg_loss:.6e}")
            
    print("Training Finished.")

In [ ]:
# --- 保存用インターフェース ---
save_button = widgets.Button(description="モデルを保存", button_style='success')
save_output = widgets.Output()

def on_save_clicked(b):
    with save_output:
        save_output.clear_output()
        # モデルだけでなく、スケーリング情報なども保存可能（今回はstate_dictのみ）
        save_data = {
            'model_state_dict': model.state_dict(),
            'config': {'nx': Config.nx, 'ny': Config.ny, 'nz': Config.nz}
        }
        torch.save(save_data, 'fem_surrogate_model.pth')
        print("モデルを 'fem_surrogate_model.pth' に保存しました。")

save_button.on_click(on_save_clicked)

if current_mode == 'train':
    print("学習結果を保存する場合はボタンを押してください:")
    display(save_button, save_output)
else:
    print("読み込みモードのため、保存ボタンは表示しません。")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch

# --- 任意荷重ケースでの3D推論 ---
# 例: 4500 N を与えて変位・ミーゼス応力を予測

# 1. 入力と同じ形のマスクを作るために、どれか1つ既存ケースのVTUを基準にする
base_vtu = vtu_files[0]  # 形状は全ケース共通想定
pts_base, disp_base, mises_base = load_case(base_vtu)

# 2. 3Dグリッド座標は学習時と同じ Config の範囲を使用
nx, ny, nz = Config.nx, Config.ny, Config.nz

# Geometry / BC / Load のマスク生成
geo_mask  = np.ones((nx, ny, nz), dtype=np.float32)              # 全体1
fix_mask  = np.zeros_like(geo_mask, dtype=np.float32); fix_mask[:, :, 0]  = 1.0   # z=0 固定
load_mask = np.zeros_like(geo_mask, dtype=np.float32); load_mask[:, :, -1] = 1.0  # z=最大に荷重

# 3. 任意荷重値（N）を指定し、学習時と同じ正規化を行う
any_load_N = 12000.0  # ★ここを変えるだけで任意荷重に対応
load_value_norm = any_load_N / Config.load_norm
val_channel = np.full_like(geo_mask, load_value_norm, dtype=np.float32)

# 4. 入力テンソルを作成 (1, nx, ny, nz, 4)
inp_np = np.stack([geo_mask, fix_mask, load_mask, val_channel], axis=-1)  # (nx, ny, nz, 4)
inp_torch = torch.from_numpy(inp_np).unsqueeze(0).to(Config.device)

# 5. 推論
model.eval()
with torch.no_grad():
    pred = model(inp_torch)  # (1, nx, ny, nz, 4)

pred_np = pred.squeeze(0).cpu().numpy()  # (nx, ny, nz, 4)

# 6. 物理量に復元（正規化の逆変換）
ux_pred_n = pred_np[..., 0]
uy_pred_n = pred_np[..., 1]
uz_pred_n = pred_np[..., 2]
mises_pred_n = pred_np[..., 3]

ux_pred = ux_pred_n * Config.scales['disp']
uy_pred = uy_pred_n * Config.scales['disp']
uz_pred = uz_pred_n * Config.scales['disp']
mises_pred = mises_pred_n * Config.scales['stress']

print(f"Prediction done for load = {any_load_N} N")
print(f"Max |Ux|: {np.max(np.abs(ux_pred)):.3e} m")
print(f"Max |Uy|: {np.max(np.abs(uy_pred)):.3e} m")
print(f"Max |Uz|: {np.max(np.abs(uz_pred)):.3e} m")
print(f"Max von Mises: {np.max(mises_pred)/1e6:.3f} MPa")

# 7. 代表スライスの簡易可視化
#   ここでは梁の x-z 断面（y 方向中央）を表示して、10x10x100 の横長形状が分かるようにする

y_mid = ny // 2  # 梁高さ方向の中央スライス

# 物理座標の範囲（Config から取得）
x_min, x_max = Config.x_range
# y は高さ方向（0〜10）だが、ここでは中央スライスなので軸には出さない
z_min, z_max = Config.z_range

# x-z 断面データを取得: shape (nx, nz)
ux_xz = ux_pred[:, y_mid, :]
uy_xz = uy_pred[:, y_mid, :]
mises_xz = mises_pred[:, y_mid, :]

fig, axes = plt.subplots(1, 3, figsize=(15, 4))
fig.suptitle(f"Predicted fields (x-z mid-y slice, Load = {any_load_N} N)")

# extent = [x_min, x_max, z_min, z_max] で横長梁を表現（x: 0→10, z: 0→100）

im0 = axes[0].imshow(
    ux_xz.T,
    origin='lower',
    cmap='jet',
    extent=[x_min, x_max, z_min, z_max],
    aspect='auto',
)
axes[0].set_title('Ux [m] (x-z, mid-y)')
axes[0].set_xlabel('x [mm]')
axes[0].set_ylabel('z [mm] (0: 固定端, 100: 荷重端)')
plt.colorbar(im0, ax=axes[0])

im1 = axes[1].imshow(
    uy_xz.T,
    origin='lower',
    cmap='jet',
    extent=[x_min, x_max, z_min, z_max],
    aspect='auto',
)
axes[1].set_title('Uy [m] (x-z, mid-y)')
axes[1].set_xlabel('x [mm]')
axes[1].set_ylabel('z [mm]')
plt.colorbar(im1, ax=axes[1])

im2 = axes[2].imshow(
    mises_xz.T,
    origin='lower',
    cmap='jet',
    extent=[x_min, x_max, z_min, z_max],
    aspect='auto',
)
axes[2].set_title('von Mises [Pa] (x-z, mid-y)')
axes[2].set_xlabel('x [mm]')
axes[2].set_ylabel('z [mm]')
plt.colorbar(im2, ax=axes[2])

plt.tight_layout()
plt.show()


モデルの読み込みと推論
上記までで作成したモデルファイルを使って、推論を行います。

In [ ]:
import torch
import os

# 保存ファイル名（保存時と同じものにしてください）
MODEL_PATH = 'fem_surrogate_model.pth'

if not os.path.exists(MODEL_PATH):
    print(f"エラー: {MODEL_PATH} が見つかりません。先に学習と保存を行ってください。")
else:
    # 1. データの読み込み
    checkpoint = torch.load(MODEL_PATH, map_location=Config.device)
    
    # 2. モデルの準備（学習時と同じ構造で初期化）
    # ※ FNO3dクラスの定義セルが既に実行されている必要があります
    loaded_model = FNO3d(
        in_channels=4, 
        out_channels=4, 
        width=32, 
        modes=8
    ).to(Config.device)
    
    # 重みの適用
    loaded_model.load_state_dict(checkpoint['model_state_dict'])
    loaded_model.eval() # 推論モードに固定
    
    print("モデルの読み込みが完了しました！推論可能です。")

In [ ]:
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np

# --- 推論用関数の定義 ---
def predict_and_visualize(force_value):
    """
    指定された荷重(N)での変位分布を予測して表示する関数
    """
    # 1. 入力データの作成 (Geometry=1, Fixed=1(z=0), Load=1(z=end))
    nx, ny, nz = Config.nx, Config.ny, Config.nz
    geo_mask = np.ones((nx, ny, nz), dtype=np.float32)
    fix_mask = np.zeros_like(geo_mask); fix_mask[:, :, 0] = 1.0
    load_mask = np.zeros_like(geo_mask); load_mask[:, :, -1] = 1.0
    
    # 荷重値の正規化 (学習時と同じ Config.load_norm で割る)
    val_channel = np.full_like(geo_mask, force_value / Config.load_norm, dtype=np.float32)
    
    # テンソル化 (Batch=1, nx, ny, nz, 4) -> (Batch, 4, nx, ny, nz) 
    # ※FNOの実装に合わせて軸を調整
    inp_np = np.stack([geo_mask, fix_mask, load_mask, val_channel], axis=-1)
    inp_tensor = torch.from_numpy(inp_np).unsqueeze(0).to(Config.device) # (1, nx, ny, nz, 4)
    
    # 2. AIによる高速推論
    with torch.no_grad():
        pred_tensor = loaded_model(inp_tensor) # (1, nx, ny, nz, 4)
    
    # 3. 結果の取り出しと物理量への復元
    pred = pred_tensor.squeeze(0).cpu().numpy()
    
    # 変位Z成分 (Uz) を取得し、スケールを戻す
    # pred[..., 2] が Uz (学習データの並び順による: ux, uy, uz, mises)
    uz_pred = pred[..., 2] * Config.scales['disp'] 
    
    # 最大変位の計算
    max_disp = np.max(np.abs(uz_pred))
    
    # --- 可視化 (断面図) ---
    # Y方向の中央断面を切り出し
    y_mid = ny // 2
    section_data = uz_pred[:, y_mid, :].T # 転置して (z, x) の向きに
    
    plt.figure(figsize=(10, 4))
    
    # ヒートマップ表示
    plt.imshow(
        section_data, 
        cmap='jet', 
        origin='lower',
        aspect='auto',
        extent=[Config.x_range[0], Config.x_range[1], Config.z_range[0], Config.z_range[1]]
    )
    plt.colorbar(label='変位 Uz [m]')
    plt.title(f"荷重: {force_value} N の時の変位分布 (予測)\n最大変位: {max_disp:.2e} m")
    plt.xlabel("X (幅) [mm]")
    plt.ylabel("Z (長さ) [mm]")
    plt.grid(False)
    plt.show()

# --- インタラクティブ・スライダーの作成 ---
print("↓ スライダーを動かして荷重を変えてみてください ↓")
widgets.interact(
    predict_and_visualize, 
    force_value=widgets.FloatSlider(
        value=-2000.0,    # 初期値
        min=-10000.0,     # 最小値
        max=0.0,          # 最大値
        step=100.0,       # 刻み幅
        description='荷重 (N):',
        layout=widgets.Layout(width='600px')
    )
);